In [2]:
from ultralytics import YOLO
import cv2

# Обучение модели
def train_model(model_name, dataset_yaml, epochs=50, imgsz=640):
    """Обучает модель YOLOv8"""
    model = YOLO(model_name)  # Загрузка предобученной модели
    model.train(data=dataset_yaml, epochs=epochs, imgsz=imgsz)
    print("Обучение завершено.")

# Детекция объектов на видео
def detect_objects_on_video(model_path, video_path, classes_to_detect):
    """Детектирует объекты на видео с использованием обученной модели"""
    model = YOLO(model_path)  # Загрузка обученной модели

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Ошибка: Не удалось получить видео.")
        return

    # Чтение параметров видео
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Настройка выходного файла
    output_video_path = 'detect.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Конец видео

        results = model.predict(frame, classes=classes_to_detect) # Выполнение детекции

        # Отображение результатов на кадре
        for result in results:  # Итерируем по списку объектов results
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0]) # Преобразование координат в целые числа
                confidence = float(box.conf[0]) # Уверенность детекции

                if confidence > 0.5:  # Фильтр по уверенности
                    class_id = int(box.cls[0])
                    class_name = result.names[class_id] # Получение имени класса из объекта results

                    # Определение цвета для каждого класса
                    color = (0, 255, 0)  # Зеленый по умолчанию
                    if class_id == 0:
                        color = (255, 0, 0)  # Красный
                    elif class_id == 1:
                        color = (0, 0, 255)  # Синий

                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)  # Рисование прямоугольника с цветом класса
                    label = f"{class_name}: {confidence:.2f}"
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Отображение кадра с результатами
        cv2.imshow("YOLOv8 Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Выход по нажатию 'q'
            break

        # Запись обработанного кадра в выходной файл
        writer.write(frame)

    cap.release()
    writer.release()
    cv2.destroyAllWindows()

In [3]:
# 1. Обучение модели (n-nano)
dataset_yaml = "dataset/data.yaml" # Путь к файлу YAML с данными
model_name = 'yolov8n.pt' # Путь к предобученной модели
train_model(model_name, dataset_yaml, epochs=50, imgsz=640)

Ultralytics 8.3.137  Python-3.10.6 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0

train: Scanning C:\Users\brink\OneDrive\Документы\yolodpp\dataset\train\labels... 50 images, 1 backgrounds, 0 corrupt: 

train: New cache created: C:\Users\brink\OneDrive\\yolodpp\dataset\train\labels.cache


val: Fast image access  (ping: 0.10.0 ms, read: 136.6285.4 MB/s, size: 112.0 KB)


val: Scanning C:\Users\brink\OneDrive\Документы\yolodpp\dataset\valid\labels... 10 images, 0 backgrounds, 0 corrupt: 10

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50      2.98G     0.6233     0.7065     0.8727          8        640: 100%|██████████| 4/4 [00:00<00:00,  4.56
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0


                   all         10         48      0.939      0.988      0.969       0.85

50 epochs completed in 0.027 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.137  Python-3.10.6 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0


                   all         10         48      0.921      0.976      0.987      0.854
                   pen         10         23      0.881      0.967       0.98       0.85
                pencil         10         25      0.961      0.986      0.993      0.858
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs\detect\train
Обучение завершено.


In [4]:
# 2. Обучение модели (s-small)
dataset_yaml = "dataset/data.yaml" # Путь к файлу YAML с данными
model_name = 'yolov8s.pt' # Путь к предобученной модели
train_model(model_name, dataset_yaml, epochs=50, imgsz=640)

Ultralytics 8.3.137  Python-3.10.6 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.

train: Scanning C:\Users\brink\OneDrive\Документы\yolodpp\dataset\train\labels.cache... 50 images, 1 backgrounds, 0 cor


val: Fast image access  (ping: 0.10.0 ms, read: 569.2151.4 MB/s, size: 112.0 KB)


val: Scanning C:\Users\brink\OneDrive\Документы\yolodpp\dataset\valid\labels.cache... 10 images, 0 backgrounds, 0 corru


Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      3.99G     0.5807       0.39     0.8633          8        640: 100%|██████████| 4/4 [00:00<00:00,  4.40
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0

                   all         10         48      0.971      0.978      0.979      0.839



50 epochs completed in 0.035 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 22.5MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.137  Python-3.10.6 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0


                   all         10         48      0.986      0.978      0.994      0.856
                   pen         10         23      0.979      0.957      0.993      0.865
                pencil         10         25      0.994          1      0.995      0.846
Speed: 0.3ms preprocess, 5.7ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs\detect\train2
Обучение завершено.


In [ ]:
# 3.1. Детекция объектов на видео (nano)
model_path = "runs/detect/train/weights/best.pt"  # Путь к обученной модели
video_path = "test.mp4" # Путь к видеофайлу
classes_to_detect = [0, 1]  # Индексы классов для детекции

detect_objects_on_video(model_path, video_path, classes_to_detect)

In [ ]:
# 3.2. Детекция объектов на видео (small)
model_path = "runs/detect/train2/weights/best.pt"  # Путь к обученной модели
video_path = "test.mp4" # Путь к видеофайлу
classes_to_detect = [0, 1]  # Индексы классов для детекции

detect_objects_on_video(model_path, video_path, classes_to_detect)

In [ ]:
# 4.1 Детекция объектов с камеры в реальном времени (nano)
model_path = "runs/detect/train/weights/best.pt"  # Путь к обученной модели
video_path = 0 # Путь к видеофайлу
classes_to_detect = [0, 1]  # Индексы классов для детекции

detect_objects_on_video(model_path, video_path, classes_to_detect)

In [ ]:
# 4.2 Детекция объектов с камеры в реальном времени (small)
model_path = "runs/detect/train/weights/best.pt"  # Путь к обученной модели
video_path = 0 # Путь к видеофайлу
classes_to_detect = [0, 1]  # Индексы классов для детекции

detect_objects_on_video(model_path, video_path, classes_to_detect)